# Switching to MediaPipe (20.12.2023)
After running into issues with the inaccuracy of Dlib when detecting which eye is closed we decided to try another model: MediaPipe. Mediapipe was developed by google and is an open-source framework that provides tools for landmark estimation. While both landmark detection and estimation identify key points on the face, landmark estimation is more precise. It provides more information about the spacial information of these points. Landmark detection on the other hand only detects if the points are on the face. Apart from a different process to analyze the face, MediaPipe also provides more than 468 landmark points. 

Another factor why MediaPipe is more precise is the attention mechanism. This is a technique modeled after the cognitive learning process of humans. The gist of it is that by selectively focusing on relevant features, the attention model can filter out irrelevant information, enabling in this case MediaPipe to prioritize the analysis of critical details. 

An important factor (though more of an added benefit than a deciding factor) is the real-time compatibility. MediaPipe is already quite lightweight making it ideal to take in our webcam inputs. This compatibility ensures that the attention model's precise analysis can be seamlessly integrated into our interactive application, where fast responses are crucial. 

| Dlib                                     | Mediapipe                         |
|------------------------------------------|-----------------------------------|
| 68 landmark points                       | approx. 468 landmark points       |
| landmark detection                       | landmark estimation               |
| not optimized for real-time applications | Real-time multimedia applications |
|                                          | attention mechanism               |